# Educational Process Mining

https://archive-beta.ics.uci.edu/dataset/346/educational+process+mining+epm+a+learning+analytics+data+set

Educational Process Mining (EPM): A Learning Analytics Data Set. (2015). UCI Machine Learning Repository.

There are 6 sessions with student input data and two excel files with intermediate and final grades. This EDA explores Session 1 data and the grade output data.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns

### Load the data from github repository

In [ ]:
inputs_path = 'https://raw.githubusercontent.com/lisaover/CMPINF2120_project/main/inputs_df.csv'
interim_path = 'https://raw.githubusercontent.com/lisaover/CMPINF2120_project/main/interim_df.csv'
final_path = 'https://raw.githubusercontent.com/lisaover/CMPINF2120_project/main/final_df.csv'
pts_path = 'https://raw.githubusercontent.com/lisaover/CMPINF2120_project/main/points_final.csv'

In [ ]:
inputs_init = pd.read_csv(inputs_path)

In [ ]:
interim_init = pd.read_csv(interim_path)

In [ ]:
final_init = pd.read_csv(final_path)

In [ ]:
pts_final_init = pd.read_csv(pts_path)

In [ ]:
inputs_init.columns

In [ ]:
interim_init.columns

In [ ]:
final_init.columns

In [ ]:
pts_final_init.columns

##### Drop Unnamed columns

In [ ]:
inputs_init.drop(columns=['Unnamed: 0'], axis=1, inplace=True)
interim_init.drop(columns=['Unnamed: 0'], axis=1, inplace=True)
final_init.drop(columns=['Unnamed: 0'], axis=1, inplace=True)
pts_final_init.drop(columns=['Unnamed: 0'], axis=1, inplace=True)

## Inputs

There are 14 columns:

* Four ID columns as type integer - session and student_id
* Five object columns, two should be dates - exercise, activity_grp, activity_ex, start_time, and end_time
* Seven integer columns - idle_time, mouse_wheel, mouse_wheel_click, mouse_click_left, mouse_click_right, mouse_movement, and keystoke

There are missing data only in the column `activity_ex` because it is a derived column from splitting the original `activity` column into two columns, one to hold the activity group and the other the exercise associated with the activity - not all activity was associated with a specific exercise.

There are 158 records where `idle_time` is negative. And, about 26% of the data has start and end times that are equal.

In [ ]:
inputs_init.info()

In [ ]:
inputs_init.isna().sum()

### Evaluate ID and date columns

In [ ]:
inputs_init.session.nunique()

In [ ]:
inputs_init.session.unique()

In [ ]:
inputs_init.student_id.nunique()

In [ ]:
inputs_init.student_id.unique()

##### Convert student_id and session to object type

In [ ]:
inputs_init['student_id'] = inputs_init['student_id'].astype('object')
inputs_init['session'] = inputs_init['session'].astype('object')

##### Convert start_time and end_time to datetime type and add variable for the total time

The variable `idle_time` is the duration of idle time between the start and end times in milliseconds.

In [ ]:
inputs_init['start_time'] = pd.to_datetime(inputs_init['start_time'])
inputs_init['end_time'] = pd.to_datetime(inputs_init['end_time'])

##### Evaluate `idle_time` and `total_time` values

`idle_time` has 158 negative values, all of which are large in magnitude with a min of -2.06 x 10^14. The average values for the mouse and keyboard actions are mostly higher for this group than those for the set of records with positive or zero idle times. 

`total_time` has one record with negative values because the start date is greater than the end date. About 26% of records have a `total_time` of 0 because the start and end times are equal. The average values for the mouse and keyboard actions are smaller for this group than for the set of records with positive or zero idle times.

In [ ]:
inputs_init['idle_time'].describe()

In [ ]:
inputs_init.loc[inputs_init['idle_time'] < 0].shape[0]

In [ ]:
inputs_init.loc[inputs_init['idle_time'] < 0].describe()

In [ ]:
inputs_init.loc[inputs_init['idle_time'] >= 0].describe()

In [ ]:
inputs_init['total_time'].describe()

In [ ]:
inputs_init.loc[inputs_init['total_time'] < 0].shape[0]

In [ ]:
inputs_init.loc[inputs_init['total_time'] < 0]

In [ ]:
inputs_init.loc[inputs_init['start_time'] > inputs_init['end_time']]

In [ ]:
inputs_init.loc[inputs_init['total_time'] == 0].describe()

In [ ]:
inputs_init.loc[inputs_init['total_time'] >= 0].describe()

In [ ]:
chk = inputs_init.copy()

In [ ]:
chk['zero_time'] = np.where(chk['start_time'] == chk['end_time'], 1, 0)

In [ ]:
chk['zero_time'].mean()

##### Change `idle_time` and `total_time` to float and assign NaN to negative values

In [ ]:
inputs_init['idle_time'] = inputs_init['idle_time'].astype('float')
inputs_init['total_time'] = inputs_init['total_time'].astype('float')

In [ ]:
inputs_init['idle_time'] = [x if x >= 0 else np.nan for x in inputs_init['idle_time']]

In [ ]:
inputs_init['total_time'] = [x if x >= 0 else np.nan for x in inputs_init['total_time']]

In [ ]:
inputs_init['idle_time'].describe()

In [ ]:
inputs_init['total_time'].describe()

##### Calculate `active_time` as difference between `total_time` and `idle_time`

In [ ]:
chk['more_idle'] = np.where(chk['idle_time'] > chk['total_time'], 1, 0)

In [ ]:
chk['more_idle'].mean()

In [ ]:
inputs_init['active_time'] = inputs_init['total_time'] - inputs_init['idle_time']

In [ ]:
inputs_init['active_time'].describe()

In [ ]:
inputs_init['active_time'] = [x if x >= 0 else np.nan for x in inputs_init['active_time']]

In [ ]:
inputs_init['active_time'].describe()

### Evaluate categorical variables

In [ ]:
inputs_init.info()

In [ ]:
inputs_init.exercise.nunique()

In [ ]:
inputs_init.exercise.unique()

In [ ]:
inputs_init.activity.nunique()

In [ ]:
inputs_init.activity_grp.unique()

In [ ]:
inputs_init.activity_grp.nunique()

In [ ]:
inputs_init.activity_grp.unique()

In [ ]:
inputs_init.activity_grp.value_counts()

In [ ]:
inputs_init.activity_ex.nunique()

In [ ]:
inputs_init.activity_ex.unique()

In [ ]:
inputs_init.activity_ex.value_counts()

### Evaluate numeric variables

In [ ]:
inputs_init.info()

##### Create functions to get summary of numeric columns

Trivial groupby column to aggregate on all rows per the following:

https://github.com/pandas-dev/pandas/issues/41768

In [ ]:
def my_q25(values):
    return np.percentile( values, q=25 )

In [ ]:
def my_q75(values):
    return np.percentile( values, q=75 )

In [ ]:
def my_iqr(values):
    return my_q75(values) - my_q25(values)

In [ ]:
def upper_outlier_bound(values): 
    return my_q75(values) + 1.5 * my_iqr(values)

In [ ]:
def lower_outlier_bound(values): 
    return my_q25(values) - 1.5 * my_iqr(values)

In [ ]:
def var_name(values):
    return values.name

In [ ]:
def outliers_exist(values):
    if (values.min() < lower_outlier_bound(values)) or (values.max() > upper_outlier_bound(values)):
        return 'Yes'
    else: 
        return 'No'

In [ ]:
def percent_outliers(values):
    df = pd.DataFrame(values)
    df['outlier'] = np.where((df[values.name] < lower_outlier_bound(values)) | (df[values.name] > upper_outlier_bound(values)), 1, 0)
    return round(df['outlier'].mean()*100, 1)

In [ ]:
def get_summary(df_in):
    
    df_out = pd.DataFrame(columns=['var_name', 'nunique', 'mean', 'median', 
                                   'min', 'lower_outlier_bound', 'max', 'upper_outlier_bound',
                                   'outliers_exist', 'percent_outliers'])
    
    # add trivial groupby column
    df_in['all'] = 'grp_all'
    
    for var in df_in.select_dtypes('number').columns.tolist():
        df_out = pd.concat([df_out, df_in.groupby(['all'])[var].\
                           aggregate( [var_name, 'nunique', 'mean', 'median', 
                                       'min', lower_outlier_bound, 'max', upper_outlier_bound, 
                                       outliers_exist, percent_outliers] ).\
                           reset_index().copy()])
        
    return df_out.drop('all', axis=1)

In [ ]:
get_summary(inputs_init.copy())

Idle_time has a negative mean. It is supposed to be the the duration of idle time between the start and end time of an activity in milliseconds.

In [ ]:
inputs_init.describe()

### Outlier Resolution

##### Create functions to support outlier resolution

In [ ]:
inputs_xdf = inputs_init.copy()

In [ ]:
def upper_extreme_bound(values):
    return np.percentile( values, q=97.5 )

In [ ]:
def lower_extreme_bound(values):
    return np.percentile( values, q=2.5 )

In [ ]:
inputs_xdf.columns

##### Natural Log

In [ ]:
inputs_xdf['total_time_ln'] = np.log(inputs_xdf[['total_time']].replace(0, np.nan))

In [ ]:
inputs_xdf['idle_time_ln'] = np.log(inputs_xdf[['idle_time']].replace(0, np.nan))

In [ ]:
inputs_xdf['mouse_wheel_ln'] = np.log(inputs_xdf[['mouse_wheel']].replace(0, np.nan))

In [ ]:
inputs_xdf['mouse_wheel_click_ln'] = np.log(inputs_xdf[['mouse_wheel_click']].replace(0, np.nan))

In [ ]:
inputs_xdf['mouse_click_left_ln'] = np.log(inputs_xdf[['mouse_click_left']].replace(0, np.nan))

In [ ]:
inputs_xdf['mouse_click_right_ln'] = np.log(inputs_xdf[['mouse_click_right']].replace(0, np.nan))

In [ ]:
inputs_xdf['mouse_movement_ln'] = np.log(inputs_xdf[['mouse_movement']].replace(0, np.nan))

In [ ]:
inputs_xdf['keystroke_ln'] = np.log(inputs_xdf[['keystroke']].replace(0, np.nan))

##### Windsorize

In [ ]:
#inputs_xdf['total_time_wdz'] = [x if x <= upper_extreme_bound(inputs_xdf['total_time']) else upper_extreme_bound(inputs_xdf['total_time']) for x in inputs_xdf['total_time']]

In [ ]:
#inputs_xdf['idle_time_wdz'] = [abs(x) if x <= upper_extreme_bound(inputs_xdf['idle_time']) else upper_extreme_bound(inputs_xdf['idle_time']) for x in inputs_xdf['idle_time']]

In [ ]:
#inputs_xdf['mouse_wheel_wdz'] = [x if x <= upper_extreme_bound(inputs_xdf['mouse_wheel']) else upper_extreme_bound(inputs_xdf['mouse_wheel']) for x in inputs_xdf['mouse_wheel']]

In [ ]:
#inputs_xdf['mouse_wheel_click_wdz'] = [x if x <= upper_extreme_bound(inputs_xdf['mouse_wheel_click']) else upper_extreme_bound(inputs_xdf['mouse_wheel_click']) for x in inputs_xdf['mouse_wheel_click']]

In [ ]:
#inputs_xdf['mouse_click_left_wdz'] = [x if x <= upper_extreme_bound(inputs_xdf['mouse_click_left']) else upper_extreme_bound(inputs_xdf['mouse_click_left']) for x in inputs_xdf['mouse_click_left']]

In [ ]:
#inputs_xdf['mouse_click_right_wdz'] = [x if x <= upper_extreme_bound(inputs_xdf['mouse_click_right']) else upper_extreme_bound(inputs_xdf['mouse_click_right']) for x in inputs_xdf['mouse_click_right']]

In [ ]:
#inputs_xdf['mouse_movement_wdz'] = [x if x <= upper_extreme_bound(inputs_xdf['mouse_movement']) else upper_extreme_bound(inputs_xdf['mouse_movement']) for x in inputs_xdf['mouse_movement']]

In [ ]:
#inputs_xdf['keystroke_wdz'] = [x if x <= upper_extreme_bound(inputs_xdf['keystroke']) else upper_extreme_bound(inputs_xdf['keystroke']) for x in inputs_xdf['keystroke']]

##### Square Root

In [ ]:
inputs_xdf['total_time_sqrt'] = np.sqrt(inputs_xdf[['total_time']])

In [ ]:
inputs_xdf['idle_time_sqrt'] = np.sqrt(inputs_xdf[['idle_time']])

In [ ]:
inputs_xdf['mouse_wheel_sqrt'] = np.sqrt(inputs_xdf[['mouse_wheel']])

In [ ]:
inputs_xdf['mouse_wheel_click_sqrt'] = np.sqrt(inputs_xdf[['mouse_wheel_click']])

In [ ]:
inputs_xdf['mouse_click_left_sqrt'] = np.sqrt(inputs_xdf[['mouse_click_left']])

In [ ]:
inputs_xdf['mouse_click_right_sqrt'] = np.sqrt(inputs_xdf[['mouse_click_right']])

In [ ]:
inputs_xdf['mouse_movement_sqrt'] = np.sqrt(inputs_xdf[['mouse_movement']])

In [ ]:
inputs_xdf['keystroke_sqrt'] = np.sqrt(inputs_xdf[['keystroke']])

##### `total_time`

In [ ]:
sns.displot(
    inputs_xdf, x="total_time", stat='percent', kind='hist', bins=30, height=4)

plt.show()

In [ ]:
sns.displot(
    inputs_xdf, x="total_time_ln", stat='percent', kind='hist', bins=30, height=4)

plt.show()

sns.displot(
    inputs_xdf, x="total_time_wdz", stat='percent', kind='hist', bins=30, height=4)

plt.show()

In [ ]:
sns.displot(
    inputs_xdf, x="total_time_sqrt", stat='percent', kind='hist', bins=30, height=4)

plt.show()

##### `idle_time`

In [ ]:
sns.displot(
    inputs_xdf, x="idle_time", stat='percent', kind='hist', bins=30, height=4)

plt.show()

In [ ]:
sns.displot(
    inputs_xdf, x="idle_time_ln", stat='percent', kind='hist', bins=30, height=4)

plt.show()

sns.displot(
    inputs_xdf, x="idle_time_wdz", stat='percent', kind='hist', bins=30, height=4)

plt.show()

##### `mouse_wheel`

In [ ]:
sns.displot(
    inputs_xdf, x="mouse_wheel", stat='percent', kind='hist', bins=30, height=4)

plt.show()

In [ ]:
sns.displot(
    inputs_xdf, x="mouse_wheel_ln", stat='percent', kind='hist', bins=30, height=4)

plt.show()

sns.displot(
    inputs_xdf, x="mouse_wheel_wdz", stat='percent', kind='hist', bins=30, height=4)

plt.show()

##### `mouse_wheel_click`

In [ ]:
sns.displot(
    inputs_xdf, x="mouse_wheel_click", stat='percent', kind='hist', bins=30, height=4)

plt.show()

In [ ]:
sns.displot(
    inputs_xdf, x="mouse_wheel_click_ln", stat='percent', kind='hist', bins=30, height=4)

plt.show()

sns.displot(
    inputs_xdf, x="mouse_wheel_click_wdz", stat='percent', kind='hist', bins=30, height=4)

plt.show()

##### `mouse_click_left`

In [ ]:
sns.displot(
    inputs_xdf, x="mouse_click_left", stat='percent', kind='hist', bins=30, height=4)

plt.show()

In [ ]:
sns.displot(
    inputs_xdf, x="mouse_click_left_ln", stat='percent', kind='hist', bins=30, height=4)

plt.show()

sns.displot(
    inputs_xdf, x="mouse_click_left_wdz", stat='percent', kind='hist', bins=30, height=4)

plt.show()

##### `mouse_click_right`

In [ ]:
sns.displot(
    inputs_xdf, x="mouse_click_right", stat='percent', kind='hist', bins=30, height=4)

plt.show()

In [ ]:
sns.displot(
    inputs_xdf, x="mouse_click_right_ln", stat='percent', kind='hist', bins=30, height=4)

plt.show()

sns.displot(
    inputs_xdf, x="mouse_click_right_wdz", stat='percent', kind='hist', bins=30, height=4)

plt.show()

##### `mouse_movement`

In [ ]:
sns.displot(
    inputs_xdf, x="mouse_movement", stat='percent', kind='hist', bins=30, height=4)

plt.show()

In [ ]:
sns.displot(
    inputs_xdf, x="mouse_movement_ln", stat='percent', kind='hist', bins=30, height=4)

plt.show()

sns.displot(
    inputs_xdf, x="mouse_movement_wdz", stat='percent', kind='hist', bins=30, height=4)

plt.show()

##### `keystroke`

In [ ]:
sns.displot(
    inputs_xdf, x="keystroke", stat='percent', kind='hist', bins=30, height=4)

plt.show()

In [ ]:
sns.displot(
    inputs_xdf, x="keystroke_ln", stat='percent', kind='hist', bins=30, height=4)

plt.show()

sns.displot(
    inputs_xdf, x="keystroke_wdz", stat='percent', kind='hist', bins=30, height=4)

plt.show()

In [ ]:
inputs_xdf.info()

In [ ]:
inputs_xdf.columns

In [ ]:
inputs_xdf.info()

In [ ]:
df_split = np.array_split(inputs_xdf, 3)

In [ ]:
pd.concat([df_split[0], df_split[1], df_split[2]], ignore_index=True, axis=0).shape

In [ ]:
df_split[0].to_csv('inputs_clean_p1.csv')

In [ ]:
df_split[1].to_csv('inputs_clean_p2.csv')

In [ ]:
df_split[2].to_csv('inputs_clean_p3.csv')

## Outputs

#### Total point values are not provided for interim session grades...assume they are all worth the same number of points. Total points for each component of the final are in the pts_final data set.

In [ ]:
interim_init.info()

In [ ]:
interim_init.rename(columns={"session_2": "interim_s2_score", "session_3": "interim_s3_score",  
                              "session_4": "interim_s4_score", "session_5": "interim_s5_score", 
                              "session_6": "interim_s6_score"}, inplace=True)

In [ ]:
interim_init.info()

In [ ]:
final_init.info()

In [ ]:
pts_final_init.info()

In [ ]:
pts_final_init.question.unique()

#### Add classification variable to indicate event 'pass_final' if composite score is greater than or equal to 70

In [ ]:
final_init['pass_final'] = np.where( final_init['final_score'] >= 70, 1, 0 )